In [ ]:
!nvidia-smi

Wed Nov 18 23:04:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Import Full-Size-Images**

In [ ]:
!gdown https://drive.google.com/uc?id=1KdpV3M27kV-_QOQOrAentfzZ2tew8YS-&

Downloading...
From: https://drive.google.com/uc?id=1KdpV3M27kV-_QOQOrAentfzZ2tew8YS-
To: /content/images_fullsize.zip
4.38GB [01:18, 55.9MB/s]


In [ ]:
!unzip -q /content/images_fullsize.zip

In [ ]:
!pip install fastai --upgrade -q
!pip install -q efficientnet_pytorch

     |████████████████████████████████| 194kB 14.0MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 


# **Libraries-Env**

In [ ]:
import random
import numpy as np
import gc
from fastai.vision.all import *
from fastai.vision import *
from pathlib import Path
import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet

In [ ]:
SEED_VAL  = 42
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [ ]:
print('Make sure cuda is installed:', torch.cuda.is_available())
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)

Make sure cuda is installed: True
Make sure cudnn is enabled: True


# **Train-Submission**

In [ ]:
def get_model(dls,pretrained=True, **kwargs):
    model = EfficientNet.from_pretrained(model_name)
    model._fc = nn.Linear(model._fc.in_features, dls.c)
    return model

In [ ]:
#params
model_name = 'efficientnet-b0'

def X_Rays_FASTAI_First_Place_Solution() :
  seed_all(SEED_VAL)
  df = pd.read_csv("/content/Train.csv")
  df2 = pd.read_csv("/content/Test.csv")
  dls = ImageDataLoaders.from_df(df,
                               path='/content', 
                               label_col="LABEL",
                               item_tfms=RandomResizedCrop(512, min_scale=0.5),
                               batch_tfms=aug_transforms(mult=1.0, do_flip=True, 
                                                        flip_vert=True, max_rotate=10., 
                                                        max_zoom=1.1,max_warp=0.2, 
                                                        p_affine=0.75, p_lighting=0, 
                                                        xtra_tfms=None),
                               valid_pct=0.03,
                               bs=8,
                               val_bs=8)
  # define - Learner
  learn = Learner(dls=dls,model= get_model(dls), metrics=[accuracy])
  print("\n `\n Start Training ....")
  learn.fit_one_cycle(20,1e-3)
  print("\n `\n Unfreeze & Again Training ....")
  learn.unfreeze()
  learn.fit_one_cycle(5,1e-4)
  print("\n `\n Start PREDICTING ....")
  test_image_ids = [img.split(".")[0] for img in os.listdir("/content/test") ]
  test_images = [f"/content/test/{img}.png" for img in test_image_ids]
  preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))
  imglabels = list(df.columns[1:])
  ls = preds[0]
  lss = list(ls[0])
  vocab = list(dls[0].dataset.vocab)
  submission = pd.DataFrame()
  submission["ID"] = test_image_ids
  for i in range(2):
    submission[vocab[i]] = ls[:, i]
  submission=submission.drop([0], axis=1)
  return submission 

In [ ]:
submission = X_Rays_FASTAI_First_Place_Solution()
submission.to_csv("effb0.csv", index=False)

Loaded pretrained weights for efficientnet-b0

 `
 Start Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.615572,0.742876,0.190476,01:26
1,0.553421,0.772796,0.285714,01:26
2,0.517613,0.968078,0.428571,01:26
3,0.528002,0.703991,0.714286,01:26
4,0.525893,1.118083,0.428571,01:26
5,0.520141,0.962780,0.714286,01:25
6,0.484368,0.304840,0.904762,01:26
7,0.489232,0.255278,0.904762,01:25
8,0.423299,0.215902,0.904762,01:26
9,0.407578,0.222195,0.904762,01:26



 `
 Unfreeze & Again Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.216248,0.092386,0.904762,01:26
1,0.214946,0.119812,0.904762,01:26
2,0.213089,0.171322,0.904762,01:26
3,0.201175,0.178572,0.904762,01:26
4,0.204729,0.174062,0.904762,01:27



 `
 Start PREDICTING ....
